In [4]:
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Dense, Activation,BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os

In [3]:
path = r"C:\Users\GUITA\Desktop\Stuff\Projects\Plant CLEF\alexnet\dataset\seg_train\seg_train"
datagen = ImageDataGenerator(validation_split=0.2, rescale=1. / 255)
train = datagen.flow_from_directory(path, subset='training', target_size=(227,227), class_mode='categorical')
test = datagen.flow_from_directory(path, subset='validation', target_size=(227,227), class_mode='categorical')

Found 5060 images belonging to 135 classes.
Found 1199 images belonging to 135 classes.


In [34]:
print("Batch Size for Input Image : ",train[0][0].shape)
print("Batch Size for Output Image : ",train[0][1].shape)
print("Image Size of first image : ",train[0][0][0].shape)
print("Output of first image : ",train[0][1][0].shape)

Batch Size for Input Image :  (32, 227, 227, 3)
Batch Size for Output Image :  (32, 135)
Image Size of first image :  (227, 227, 3)
Output of first image :  (135,)


In [12]:
X_input = Input(train[0][0].shape[1:])
    
X = Conv2D(96,(11,11),strides = 4,name="conv0")(X_input)
#X = Dropout(0.2, input_shape=(96,))(X)
X = BatchNormalization(axis = 3 , name = "bn0")(X)
X = Activation('relu')(X)
    
X = MaxPooling2D((3,3),strides = 2,name = 'max0')(X)
    
X = Conv2D(256,(5,5),padding = 'same' , name = 'conv1')(X)
#X = Dropout(0.2, input_shape=(256,))(X)
X = BatchNormalization(axis = 3 ,name='bn1')(X)
X = Activation('relu')(X)
    
X = MaxPooling2D((3,3),strides = 2,name = 'max1')(X)
    
X = Conv2D(384, (3,3) , padding = 'same' , name='conv2')(X)
#X = Dropout(0.2, input_shape=(384,))(X)
X = BatchNormalization(axis = 3, name = 'bn2')(X)
X = Activation('relu')(X)
    
X = Conv2D(384, (3,3) , padding = 'same' , name='conv3')(X)
#X = Dropout(0.2, input_shape=(384,))(X)
X = BatchNormalization(axis = 3, name = 'bn3')(X)
X = Activation('relu')(X)
    
X = Conv2D(256, (3,3) , padding = 'same' , name='conv4')(X)
#X = Dropout(0.5, input_shape=(256,))(X)
X = BatchNormalization(axis = 3, name = 'bn4')(X)
X = Activation('relu')(X)
    
X = MaxPooling2D((3,3),strides = 2,name = 'max2')(X)
    
X = Flatten()(X)
    
X = Dense(4096, activation = 'relu', name = "fc0")(X)

X = Dense(4096, activation = 'relu', name = 'fc1')(X) 

X = Dense(135,activation='softmax',name = 'fc2')(X)

model = Model(inputs = X_input, outputs = X, name='AlexNet')


In [9]:
model.summary()

Model: "AlexNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 conv0 (Conv2D)              (None, 55, 55, 96)        34944     
                                                                 
 bn0 (BatchNormalization)    (None, 55, 55, 96)        384       
                                                                 
 activation_5 (Activation)   (None, 55, 55, 96)        0         
                                                                 
 max0 (MaxPooling2D)         (None, 27, 27, 96)        0         
                                                                 
 conv1 (Conv2D)              (None, 27, 27, 256)       614656    
                                                                 
 bn1 (BatchNormalization)    (None, 27, 27, 256)       1024

In [13]:
opt = tf.optimizers.Adagrad(learning_rate=0.00495, epsilon=None, decay=0.0)
model.compile(optimizer = opt , loss = tf.keras.losses.KLDivergence() , metrics=['accuracy'])

In [8]:
from keras.optimizers import gradient_descent_v2

In [7]:
opt = gradient_descent_v2.SGD(learning_rate=2e-2, momentum=0.9, decay=1e-2/20)
model.compile(optimizer = opt , loss = 'categorical_crossentropy' , metrics=['accuracy'])

NameError: name 'gradient_descent_v2' is not defined

In [14]:
model.fit(train,epochs=20)

Epoch 1/20
159/159 [==============================] - 176s 1s/step - loss: 4.5821 - accuracy: 0.0543
Epoch 2/20
159/159 [==============================] - 175s 1s/step - loss: 3.7861 - accuracy: 0.1217
Epoch 3/20
159/159 [==============================] - 180s 1s/step - loss: 3.4003 - accuracy: 0.1889
Epoch 4/20
159/159 [==============================] - 174s 1s/step - loss: 3.0553 - accuracy: 0.2407
Epoch 5/20
159/159 [==============================] - 195s 1s/step - loss: 2.6516 - accuracy: 0.3221
Epoch 6/20
159/159 [==============================] - 195s 1s/step - loss: 2.2986 - accuracy: 0.4053
Epoch 7/20
159/159 [==============================] - 198s 1s/step - loss: 1.8544 - accuracy: 0.5115
Epoch 8/20
159/159 [==============================] - 185s 1s/step - loss: 1.5071 - accuracy: 0.6219
Epoch 9/20
159/159 [==============================] - 192s 1s/step - loss: 1.0915 - accuracy: 0.7225
Epoch 10/20
159/159 [==============================] - 181s 1s/step - loss: 0.8027 - accura

In [15]:
preds=model.evaluate(test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

38/38 [==============================] - 10s 246ms/step - loss: 3.8766 - accuracy: 0.2819
Loss = 3.876593828201294
Test Accuracy = 0.28190159797668457
